In [1]:
import os
# import constants
# from datasets import load_dataset
# from utils import clean, make_current_datetime_dir, compute_metrics, preprocess_data

import pandas as pd
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import TrainingArguments, Trainer, logging

import torch



2024-05-14 09:48:21.420663: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-14 09:48:21.420730: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-14 09:48:21.420764: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-14 09:48:21.430613: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="5"

# CUDA_DEVICE_ORDER 및 CUDA_VISIBLE_DEVICES 환경 변수 확인
cuda_order = os.getenv("CUDA_DEVICE_ORDER")
cuda_visible_devices = os.getenv("CUDA_VISIBLE_DEVICES")

print("CUDA_DEVICE_ORDER:", cuda_order)
print("CUDA_VISIBLE_DEVICES:", cuda_visible_devices)

print(torch.__version__)
# device = "cpu"
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

CUDA_DEVICE_ORDER: PCI_BUS_ID
CUDA_VISIBLE_DEVICES: 5
1.13.1+cu117
Using cuda device


In [3]:
model_path = './temp/electra'
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [4]:
# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir='./results',  # 결과 및 임시 파일 저장 경로
    do_train=False,  # 훈련 안 함
    do_predict=True  # 예측 수행
)

# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args
)


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [5]:
class KoelectraDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [6]:

# tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base-v2022")
tokenizer = AutoTokenizer.from_pretrained("./tokenizer")

test_path = "mydata/my_pred.csv"
test_data = pd.read_csv(test_path, sep=',')

max_len = 64          # 문장의 길이 (평균 13정도)
batch_size = 32
num_epochs = 3
log_interval = 500    # metrics 생성 시점



In [7]:
# 테스트 데이터
# ==> 토크나이징 + 패딩 : dim=32
encoded_test = tokenizer(
    test_data['text'].tolist(),
    return_tensors='pt',
    max_length=max_len,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

print('디코딩 :',tokenizer.decode(encoded_test[0].ids))
print('디코딩 :',tokenizer.decode(encoded_test[-1].ids))


디코딩 : [CLS] 유저0 : 원조가 진짜 생존 유튜버였고 그 유튜브 보면 관련 영상으로 짭이 뜨는데 제가 처음 그거 봤을 때 영상 속에서 중장비 캐터필러 자국을 발견해서 단박에 주작인지 눈치챘었음ㅋㅋㅋ 그리고 다른 짭들도 작업속도가 지나치게 빨라서 다 주작이구나 했는데 결국 현장 [SEP]
디코딩 : [CLS] # # # [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [8]:
# https://huggingface.co/course/en/chapter3/3?fw=pt#evaluation
# 데이터셋 생성
zero_labels = np.zeros(len(test_data), dtype=int)
test_dataset = KoelectraDataset(encoded_test, zero_labels)
# 예측하기
predictions = trainer.predict(test_dataset)
print(predictions.predictions.shape, predictions.label_ids.shape)

/tmp/ipykernel_2748402/548597468.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


(672, 3) (672,)


In [9]:
from scipy.special import softmax

probabilities = softmax(predictions.predictions, axis=1)
predicted_labels = np.argmax(probabilities, axis=1)


In [10]:
test_texts = test_data['text']  # 테스트 데이터셋에서 텍스트 열 추출
actual_labels = test_data['label']  # 실제 레이블 열

df_results = pd.DataFrame({
    "text": test_texts,
    "actual_label": actual_labels,
    "predicted_label": predicted_labels,
    "probability_0": probabilities[:, 0],
    "probability_1": probabilities[:, 1],
    "probability_2": probabilities[:, 2]  # 클래스가 더 많다면 이를 확장
})


In [11]:
# 예측이 틀린 경우만 보기
incorrect_predictions = df_results[df_results['actual_label'] != df_results['predicted_label']]
correct_predictions = df_results[df_results['actual_label'] == df_results['predicted_label']]
# print(incorrect_predictions)
incorrect_predictions

,text,actual_label,predicted_label,probability_0,probability_1,probability_2
17,유저16: ㅋㅋㅋㅋㅋㅋㅋㅋ재밌노 고소한다 그러노 근송이 큰일났네,1.0,2,0.001301,0.000774,0.997925
18,유저14: @털복이 솔직히 나이 어린 게 좀 티나긴 해...ㅋㅋ,2.0,1,0.002178,0.669407,0.328415
32,유저28: @안근 송 나 병@인가봐.....,2.0,0,0.966252,0.004216,0.029531
52,유저47: 그것들을 진짜라고 생각했으면 ㄹㅇ 그런 지능으로 어떻게 살아가노,0.0,2,0.033828,0.018019,0.948153
53,유저48: 하 병신의 나라답다. 주작이라고 밝혀지니까 너도나도 애초에 수상했데 ㅋㅋ...,2.0,0,0.959671,0.011670,0.028659
...,...,...,...,...,...,...
648,유저7: @@xhahrbhtahdhhd 개꼬였네,0.0,2,0.011155,0.004136,0.984708
649,유저8: @@xhahrbhtahdhhd 누가 그 유튜버가 혼자 구석기-신석기~청동기...,0.0,2,0.037407,0.013594,0.948999
650,유저9: 이게 맞지. 애초에 그 사람은 생존 보단 원시의 기술을 재현하는 데에 의의...,2.0,0,0.514852,0.009345,0.475802
652,유저11: @@user-bn9tv2ds1i 애니프사,1.0,2,0.001591,0.000924,0.997485


In [12]:
accuracy = len(correct_predictions) / len(test_data)


print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8586


In [13]:
incorrect_predictions[incorrect_predictions['actual_label']==2]

,text,actual_label,predicted_label,probability_0,probability_1,probability_2
18,유저14: @털복이 솔직히 나이 어린 게 좀 티나긴 해...ㅋㅋ,2.0,1,0.002178,0.669407,0.328415
32,유저28: @안근 송 나 병@인가봐.....,2.0,0,0.966252,0.004216,0.029531
53,유저48: 하 병신의 나라답다. 주작이라고 밝혀지니까 너도나도 애초에 수상했데 ㅋㅋ...,2.0,0,0.959671,0.011670,0.028659
66,유저57: @@user-bo1zx8jb6q 넌 황현진빠는 무지성이니까,2.0,1,0.008104,0.960363,0.031533
84,유저75: 당연한거지n중장비로 하루 투자해서n사람이 한달동안ㅈ빠지게 해야하는일을n금...,2.0,0,0.672207,0.009675,0.318118
183,유저24: @@iple4445 문법 나치같이 한 것도 아니고 그냥 알려줘도 지랄이네...,2.0,0,0.976921,0.001239,0.021840
187,유저28: 아이고ㅆㅂ 맞춤범 지적안하면 손가락이 부러지나?그토록 십선인척 하고 싶었...,2.0,0,0.902007,0.006001,0.091992
199,유저40: @@neongnyang 녕냥갱 먹고싶누,2.0,0,0.945557,0.043988,0.010455
223,유저60: 다름아닌 그깟 맞춤법 하나 가지고 이렇게 싸우는 거냐 지금? 역시 사소한...,2.0,0,0.826851,0.062747,0.110402
248,유저52: @@user-vq2rk8vs4f 네 그건 됐구여 왜 국립국어원에서 며칠이...,2.0,0,0.974608,0.001573,0.023819


In [14]:
# pd.set_option('display.max_rows', 100)

incorrect_predictions[incorrect_predictions['predicted_label']==2]

,text,actual_label,predicted_label,probability_0,probability_1,probability_2
17,유저16: ㅋㅋㅋㅋㅋㅋㅋㅋ재밌노 고소한다 그러노 근송이 큰일났네,1.0,2,0.001301,0.000774,0.997925
52,유저47: 그것들을 진짜라고 생각했으면 ㄹㅇ 그런 지능으로 어떻게 살아가노,0.0,2,0.033828,0.018019,0.948153
93,유저84: 주작이든 아니든 크리에이터 본인이 영상만드는건데 뭔 범죄라도 저지른것마냥...,1.0,2,0.001906,0.000893,0.997201
102,유저93: 구라까네 몰랐으면서,0.0,2,0.074731,0.009342,0.915927
104,유저93: @@ttulgi09 현장에서 일하는 사람말고는 다 알았다는 사람들은 개구...,0.0,2,0.012768,0.004934,0.982297
...,...,...,...,...,...,...
602,유저5: @@bearangry9075 억지국뽕,1.0,2,0.002753,0.400347,0.596900
648,유저7: @@xhahrbhtahdhhd 개꼬였네,0.0,2,0.011155,0.004136,0.984708
649,유저8: @@xhahrbhtahdhhd 누가 그 유튜버가 혼자 구석기-신석기~청동기...,0.0,2,0.037407,0.013594,0.948999
652,유저11: @@user-bn9tv2ds1i 애니프사,1.0,2,0.001591,0.000924,0.997485


In [15]:
# tokenizer.save_pretrained('./tokenizer')
# torch.save(model.state_dict(), 'model.pth')